# Image Morphing 
The goal of this exercise is to present a simple method for image morphing. We will work with pairs of images containing faces which are annotated with facial landmarks. The goal is to morph (transform) a face into another other face by creating a smooth transition between the source image and the destination image. The transition contains fake images obtained using geometric and photometric interpolation based on regions (triangles) selected from the two images. The method can be easily extended to other class of objects (different than faces).

<img src="images\final_result.gif" width=100 />

In achieving our goal, we will follow the next steps:
1. compute the Delauney triangulation based on the facial landmarks in the source image, partinioning the face in triangles. We transfer this triangulation to the destination image for corresponding facial landmarks;
2. compute the affine transformation between pairs of corresponding triangles from the two images;
3. create fake triangles based on geometric and photometric interpolation of triangles selected  from the source and destination images

<table width="950px">
<tr>
<th><center>Source - real</center></th>
<th><center>fake</center></th> 
<th><center>fake</center></th> 
<th><center>fake</center></th> 
<th><center>fake</center></th> 
<th><center>fake</center></th> 
<th><center>Destination - real</center></th> 
</tr>
<tr>
<td><img src="images\img_0.jpg" width=100 /></td>
<td><img src="images\img_10.jpg" width=100 /></td>
<td><img src="images\img_20.jpg" width=100 /></td>
<td><img src="images\img_30.jpg" width=100 /></td>
<td><img src="images\img_40.jpg" width=100 /></td>
<td><img src="images\img_50.jpg" width=100 /></td>
<td><img src="images\img_60.jpg" width=100 /></td> 
</tr>
</table>   

In [1]:
import numpy as np
import cv2 as cv
from scipy.spatial import Delaunay
import scipy.io

In [2]:
def show_image_points(img,points):    
    img_ = img.copy()
    for p in points:
        cv.circle(img_, (p[0], p[1]), 3, (0, 255, 0), -1)    
    cv.imshow("image",img_)
    cv.waitKey(0) # waits until a key is pressed
    cv.destroyAllWindows() # destroys the window showing image
    return img_

In [3]:
#show the annotated facial landmarks on each image
img1 = cv.imread('images/bogdan_crop.jpg'); #source image
temp = scipy.io.loadmat('data/bogdan_points.mat')
p1 = temp['points']
p1 = np.uint64(p1)
img1_ = show_image_points(img1, p1)

img2 = cv.imread('images/clooney_crop.jpg');#destination image
temp = scipy.io.loadmat('data/clooney_points.mat')
p2 = temp['points']
p2 = np.uint64(p2)
img2_ = show_image_points(img2, p2)

In [7]:
cv.imshow("img1_",img1_)
cv.imshow("img2_",img2_)
cv.waitKey(0)
cv.destroyAllWindows()

In [4]:
#compute the Delauney triangulation based on the facial landmarks in the source image
#the face is partitioned in triangles
triangulation = Delaunay(p1)
print("We have obtained ", str(triangulation.simplices.shape[0]), " triangles")
print(triangulation.simplices) #show for each triangle the index of each vertex

We have obtained  70  triangles
[[18 17 19]
 [41 12  0]
 [20  7 19]
 [ 7 18 19]
 [ 9 24 10]
 [40 41  0]
 [41 40 38]
 [41 42 12]
 [42 41 38]
 [ 6  5 16]
 [17  6 16]
 [18  6 17]
 [ 7  6 18]
 [34 25 29]
 [27 25 34]
 [ 8  7 20]
 [ 8 31  7]
 [ 8 20 21]
 [31  8 21]
 [31 33  7]
 [33  6  7]
 [35 34 29]
 [33 35 29]
 [35 36 38]
 [36 32 10]
 [32  9 10]
 [32 36 26]
 [31 32 26]
 [ 9 32 22]
 [32 31 21]
 [22 32 21]
 [ 9 23 24]
 [23  9 22]
 [23 22 21]
 [ 1 40  0]
 [40 37 38]
 [37 35 38]
 [35 37 34]
 [ 1 37 40]
 [ 2 37  1]
 [42 11 12]
 [24 11 10]
 [36 39 38]
 [39 42 38]
 [39 11 42]
 [39 36 10]
 [11 39 10]
 [14  4 13]
 [ 4 27 34]
 [27 15  5]
 [ 5 15 16]
 [15 14 16]
 [ 4 15 27]
 [15  4 14]
 [33 28  6]
 [ 6 28  5]
 [25 28 29]
 [28 33 29]
 [28 27  5]
 [28 25 27]
 [36 30 26]
 [30 31 26]
 [30 33 31]
 [35 30 36]
 [30 35 33]
 [ 3  2 13]
 [ 3 37  2]
 [37  3 34]
 [ 4  3 13]
 [ 3  4 34]]


In [5]:
#plot the triangles partitioning the face, each triangle with a different color
def show_image_triangulated(img,points,triangulation):
    img_t = img.copy()
    colors = [[255,0,0],[0,255,0],[0,0,255],[255,255,0],[255,0,255],[0,255,255],[0,0,0],[128,255,255],[255,128,255],[255,255,128]]
    number_colors = len(colors)    
    
    for idx,t in enumerate(triangulation.simplices):
        point1 = points[t[0]]
        #cv.circle(img, (point1[0], point1[1]), 3, (0, 255, 0), -1)
        point2 = points[t[1]]
        #cv.circle(img, (point2[0], point2[1]), 3, (0, 255, 0), -1)
        point3 = points[t[2]]
        #cv.circle(img, (point3[0], point3[1]), 3, (0, 255, 0), -1)
        
        cv.line(img_t, (point1[0],point1[1]), (point2[0],point2[1]), colors[idx % number_colors], 3)
        cv.line(img_t, (point2[0],point2[1]), (point3[0],point3[1]), colors[idx % number_colors], 3)
        cv.line(img_t, (point3[0],point3[1]), (point1[0],point1[1]), colors[idx % number_colors], 3)            
        
    cv.imshow("image with triangles",img_t)
    cv.waitKey(0) # waits until a key is pressed
    cv.destroyAllWindows() # destroys the window showing image
    return img_t

img_t_1 = show_image_triangulated(img1,p1,triangulation)
#transfer the triangulation to the destination image (we migh have triangles which overlap)
img_t_2 = show_image_triangulated(img2,p2,triangulation)

In [6]:
cv.imshow("image1",img_t_1)
cv.imshow("image2",img_t_2)
cv.waitKey(0)
cv.destroyAllWindows()

Compute the affine transformation T between two triangles ABC and DEF using the formula derived in lecture 9. We use the proxy triangle with vertices (0,1), (0,0) and (1,0) to get affine transformation T1 and T2. Then, we have that T = T2 * inv(T1)
<table width="950px">
<tr>
<td><img src="images\affine_transformation_1.png" width=450 /></td>
<td><img src="images\affine_transformation_2.png" width=450 /></td>    
</tr>
</table>   

In [7]:
def compute_affine_transformation(triangle_1, triangle_2):# 
# find the affine transformation matrix T that transforms vertices of triangle_1 in vertices of triangle_2
# follow the lecture: compute first T1, T2, than T = T2 * inv(T1) 
    A = triangle_1[0]
    B = triangle_1[1]
    C = triangle_1[2]    
    
    T1 = np.zeros((3,3))
    T1[0,:] = [C[0]-A[0],B[0]-A[0],A[0]]
    T1[1,:] = [C[1]-A[1],B[1]-A[1],A[1]]
    T1[2,:] = [0,0,1]
    
    D = triangle_2[0]
    E = triangle_2[1]
    F = triangle_2[2]
    
    T2 = np.zeros((3,3))
    T2[0,:] = [F[0]-D[0],E[0]-D[0],D[0]]
    T2[1,:] = [F[1]-D[1],E[1]-D[1],D[1]]
    T2[2,:] = [0,0,1]
        
    T = T2 @ np.linalg.inv(T1)
        
    return T

In [8]:
#sanity check for the first triangle with vertices = points 18, 17, 19

#take triangle ABC: A = p1[18], B = p1[17], C = p1[19]
triangle_1 = np.float32([p1[18],p1[17],p1[19]])

#take triangle DEF: D= p2[18], E = p2[17], F = p2[19]
triangle_2 = np.float32([p2[18],p2[17],p2[19]])

print("Triangle ABC: ", triangle_1,"\n")
print("Triangle DEF:", triangle_2,"\n")

T = compute_affine_transformation(triangle_1,triangle_2)
print("The affine transformation matrix T = ", T,"\n")

#verify that T transforms A in D
print("Verifying that T trasforms A in D: A = ",p1[18], " D = ", p2[18])
print("Apply T to point A. We obtain ",T.dot([p1[18][0],p1[18][1],1]), "\n")

#verify that T transforms B in E
print("Verifying that T transforms B in E: B = ",p1[17], " E = ", p2[17])
print("Apply T to point B. We obtain ",T.dot([p1[17][0],p1[17][1],1]),"\n")

#verify that T transforms C in F
print("Verifying that T transforms C in F: C = ",p1[19], " F = ", p2[19])
print("Apply T to point C. We obtain ",T.dot([p1[19][0],p1[19][1],1]),"\n")

#verify that T transforms middle point of AB in middle point of DE
mAB = 0.5*p1[17] + 0.5*p1[18]  
mDE = 0.5*p2[17] + 0.5*p2[18]
print("middle point of AB = ", mAB)
print("middle points of DE = ", mDE)
print("Verifying that T transforms middle point of AB in middle point of DE: mAB = ",mAB, " mDE = ", mDE)
print("Apply T to point mAB. We obtain ",T.dot([mAB[0],mAB[1],1]),"\n")

Triangle ABC:  [[231.  24.]
 [157.  13.]
 [332.  33.]] 

Triangle DEF: [[255.   4.]
 [169.   4.]
 [354.  27.]] 

The affine transformation matrix T =  [[  0.70786517   3.05617978  18.13483146]
 [  0.56853933  -3.8247191  -35.53932584]
 [  0.           0.           1.        ]] 

Verifying that T trasforms A in D: A =  [231  24]  D =  [255   4]
Apply T to point A. We obtain  [255.   4.   1.] 

Verifying that T transforms B in E: B =  [157  13]  E =  [169   4]
Apply T to point B. We obtain  [169.   4.   1.] 

Verifying that T transforms C in F: C =  [332  33]  F =  [354  27]
Apply T to point C. We obtain  [354.  27.   1.] 

middle point of AB =  [194.   18.5]
middle points of DE =  [212.   4.]
Verifying that T transforms middle point of AB in middle point of DE: mAB =  [194.   18.5]  mDE =  [212.   4.]
Apply T to point mAB. We obtain  [212.   4.   1.] 



In [9]:
# Warps and alpha blends triangular regions from img1 and img2 to img
def morph_triangles(img1, img2, img_morphed, t1, t2, alpha,show_images) :

    #compute the coordinates of the triangle in the morphed image as average weighting
    t = (1-alpha) * t1 + alpha * t2
    
    # Find bounding rectangle for each triangle
    r1 = cv.boundingRect(np.float32([t1]))
    r2 = cv.boundingRect(np.float32([t2]))
    r = cv.boundingRect(np.float32([t]))

    # Offset points by left top corner of the respective rectangles
    t1_rect = []
    t2_rect = []
    t_rect = []

    for i in range(0, 3):
        t_rect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1_rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2_rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv.fillConvexPoly(mask, np.int32(t_rect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1_rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2_rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    
    T = compute_affine_transformation(t1_rect, t_rect)        
    warp_img1 = cv.warpAffine(img1_rect, T[:2], (size[0], size[1]), None, flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REFLECT_101 )
    
    T = compute_affine_transformation(t2_rect, t_rect)    
    warp_img2 = cv.warpAffine(img2_rect, T[:2], (size[0], size[1]), None, flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REFLECT_101 )
       

    # Alpha blend rectangular patches
    img_rect = np.uint8((1.0 - alpha) * warp_img1 + alpha * warp_img2)
    
    # Copy triangular region of the rectangular patch to the output image
    img_morphed[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img_morphed[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + img_rect * mask
    
    if show_images:
        cv.imshow("img1_Rect",img1_rect)
        cv.imshow("warped image 1",warp_img1)
        cv.imshow("img2_Rect",img2_rect)
        cv.imshow("warped image 2",warp_img2)
        cv.imshow("blend rectangular patches",img_rect)
        cv.imshow("mask",mask)
        cv.imshow("img_morphed",img_morphed) 
        cv.waitKey(0)
        cv.destroyAllWindows()
                
    return img_morphed

In [10]:
#do a demo for just one triangle
#morph a triangle to the corresponding triangle
#take the pairs of triangles from line 20, with vertices the landmarks: 31 33  7

#take triangle ABC: A = p1[31], B = p1[33], C = p1[7]
t1 = np.float32([p1[31],p1[33],p1[7]])

#take triangle DEF: D= p2[31], E = p2[33], F = p2[7]
t2 = np.float32([p2[31],p2[33],p2[7]])

alpha = 0.5
img_morphed = np.zeros(img1.shape, dtype = img1.dtype)
img_morphed = morph_triangles(img1, img2, img_morphed, t1, t2, alpha,1)

In [15]:
def morph_image(img1,img2,p1,p2,triangulation,alpha,show_images):
    
    img_morph = np.zeros(img1.shape, dtype = img1.dtype)
    
    #fix the background
    #img_morph = (1-alpha) * img1 + alpha * img2
    
    for idx,t in enumerate(triangulation.simplices):       
               
        A = p1[t[0]]
        B = p1[t[1]]
        C = p1[t[2]]    
        ABC = np.array([A,B,C])        
        #ABC = [A,B,C]
    
        D = p2[t[0]]
        E = p2[t[1]]
        F = p2[t[2]]
        DEF = np.array([D,E,F])
            
        # warps and alpha blends triangular regions ABC from img1 and DEF from img2 to MNO from img
        img_morph = morph_triangles(img1, img2, img_morph, ABC, DEF, alpha,show_images)
        
    return img_morph

In [16]:
img1 = cv.imread('images/bogdan_crop.jpg'); #source image
img2 = cv.imread('images/clooney_crop.jpg');#destination image

temp = scipy.io.loadmat('data/bogdan_points.mat')
p1 = temp['points']
p1 = np.uint64(p1)

temp = scipy.io.loadmat('data/clooney_points.mat')
p2 = temp['points']
p2 = np.uint64(p2)

triangulation = Delaunay(p1)
number_steps = 50

for step in range(0,number_steps+1):
    alpha = step/number_steps
    
    #print("alpha = ", alpha)
    img_morph = morph_image(img1, img2, p1, p2, triangulation, alpha,0)
    
    # Display Result
    cv.imwrite("gif/image_" + str(step) +  ".png",img_morph)
    cv.imshow("Morphed Face", np.uint8(img_morph))
    cv.waitKey(500)
    cv.destroyAllWindows()

In [ ]:
#go to https://gifmaker.me/ for making a gif
#could also install some libraries to make gif in opencv